## SQLITE DATABASE

## Define Paths & Match Formats

In [1]:
import json
import sqlite3
from pathlib import Path
from tqdm import tqdm

# Base data folder
base_path = Path("data")

# Match formats and their folders
formats = {
    "Test": base_path / "tests_json",
    "ODI": base_path / "odis_json",
    "T20I": base_path / "it20s_json"
}

# SQLite DB
db_path = "cricsheet.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

## Create Normalized Tables

In [2]:
# Teams
cursor.execute("""
CREATE TABLE IF NOT EXISTS teams (
    team_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE
)
""")

# Players
cursor.execute("""
CREATE TABLE IF NOT EXISTS players (
    player_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE
)
""")

# Matches
cursor.execute("""
CREATE TABLE IF NOT EXISTS matches (
    match_id TEXT PRIMARY KEY,
    format TEXT,
    city TEXT,
    date TEXT,
    venue TEXT,
    toss_winner_id INTEGER,
    toss_decision TEXT,
    result TEXT,
    winner_id INTEGER,
    FOREIGN KEY (toss_winner_id) REFERENCES teams(team_id),
    FOREIGN KEY (winner_id) REFERENCES teams(team_id)
)
""")

# Innings
cursor.execute("""
CREATE TABLE IF NOT EXISTS innings (
    inning_id INTEGER PRIMARY KEY AUTOINCREMENT,
    match_id TEXT,
    inning_number INTEGER,
    batting_team_id INTEGER,
    FOREIGN KEY (match_id) REFERENCES matches(match_id),
    FOREIGN KEY (batting_team_id) REFERENCES teams(team_id)
)
""")

# Deliveries
cursor.execute("""
CREATE TABLE IF NOT EXISTS deliveries (
    delivery_id INTEGER PRIMARY KEY AUTOINCREMENT,
    inning_id INTEGER,
    over INTEGER,
    ball INTEGER,
    batsman_id INTEGER,
    bowler_id INTEGER,
    runs INTEGER,
    extras INTEGER,
    wicket_type TEXT,
    FOREIGN KEY (inning_id) REFERENCES innings(inning_id),
    FOREIGN KEY (batsman_id) REFERENCES players(player_id),
    FOREIGN KEY (bowler_id) REFERENCES players(player_id)
)
""")

## Utility Functions

In [3]:
def get_or_create_team(name):
    cursor.execute("SELECT team_id FROM teams WHERE name = ?", (name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    cursor.execute("INSERT INTO teams (name) VALUES (?)", (name,))
    return cursor.lastrowid

def get_or_create_player(name):
    cursor.execute("SELECT player_id FROM players WHERE name = ?", (name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    cursor.execute("INSERT INTO players (name) VALUES (?)", (name,))
    return cursor.lastrowid

## Ingest JSON Files

In [4]:
def insert_match(file_path: Path, match_format: str):
    match_id = file_path.stem
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    info = data.get("info", {})
    teams = info.get("teams", ["", ""])
    team1_id = get_or_create_team(teams[0])
    team2_id = get_or_create_team(teams[1])
    toss_winner_id = get_or_create_team(info.get("toss", {}).get("winner", ""))
    winner_id = get_or_create_team(info.get("outcome", {}).get("winner", ""))

    cursor.execute("""
    INSERT OR REPLACE INTO matches VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        match_id,
        match_format,
        info.get("city"),
        info.get("dates", [""])[0],
        info.get("venue"),
        toss_winner_id,
        info.get("toss", {}).get("decision"),
        info.get("outcome", {}).get("result"),
        winner_id
    ))

    # Insert innings and deliveries
    for inning_num, inning in enumerate(data.get("innings", []), start=1):
        batting_team_id = get_or_create_team(inning.get("team"))
        cursor.execute("""
        INSERT INTO innings (match_id, inning_number, batting_team_id)
        VALUES (?, ?, ?)
        """, (match_id, inning_num, batting_team_id))
        inning_id = cursor.lastrowid

        for over_data in inning.get("overs", []):
            over = over_data.get("over")
            for delivery in over_data.get("deliveries", []):
                ball = delivery.get("ball")
                batsman_id = get_or_create_player(delivery.get("batter"))
                bowler_id = get_or_create_player(delivery.get("bowler"))
                runs = delivery.get("runs", {}).get("batter", 0)
                extras = delivery.get("runs", {}).get("extras", 0)
                wicket_type = delivery.get("wickets", [{}])[0].get("kind") if delivery.get("wickets") else None

                cursor.execute("""
                INSERT INTO deliveries (inning_id, over, ball, batsman_id, bowler_id, runs, extras, wicket_type)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                """, (
                    inning_id, over, ball, batsman_id, bowler_id, runs, extras, wicket_type
                ))

## Run Ingestion


In [5]:
for match_format, folder in formats.items():
    print(f"Processing {match_format} matches...")
    for file_path in tqdm(list(folder.glob("*.json"))):
        try:
            insert_match(file_path, match_format)
        except Exception as e:
            print(f"Error in {file_path.name}: {e}")

conn.commit()
conn.close()
print("All matches inserted into normalized SQLite database!")

Processing Test matches...


100%|██████████| 878/878 [00:48<00:00, 18.29it/s]


Processing ODI matches...


100%|██████████| 2988/2988 [01:06<00:00, 45.17it/s]


Processing T20I matches...


100%|██████████| 320/320 [00:04<00:00, 72.19it/s]


All matches inserted into normalized SQLite database!
